In [28]:
import random
import pickle
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import scipy.misc
import cv2

In [29]:
# Here is the section dedicated to contatain the results of the sampling
# Purpose here is to try and use the UNET architecture and understand the
# first steps of building and training the neural network
sampled_pickles = '/home/omar/Desktop/sampled_output/'
smooth = 1.0


patients = os.listdir(sampled_pickles)
random.shuffle(patients)
# training_patients = patients[:26]
training_patients = ['TCGA-76-6282.pickle', 'TCGA-76-4932.pickle', 'TCGA-02-0046.pickle', 'TCGA-06-0138.pickle', 'TCGA-76-6657.pickle', 'TCGA-76-4935.pickle', 'TCGA-19-5960.pickle', 'TCGA-76-6280.pickle', 'TCGA-02-0006.pickle', 'TCGA-19-1789.pickle', 'TCGA-02-0009.pickle', 'TCGA-19-0963.pickle', 'TCGA-02-0086.pickle', 'TCGA-06-0149.pickle', 'TCGA-06-0179.pickle', 'TCGA-06-2570.pickle', 'TCGA-02-0075.pickle', 'TCGA-12-0616.pickle', 'TCGA-02-0085.pickle', 'TCGA-06-5417.pickle', 'TCGA-06-0145.pickle', 'TCGA-08-0355.pickle', 'TCGA-02-0054.pickle', 'TCGA-19-2624.pickle', 'TCGA-06-0177.pickle', 'TCGA-02-0106.pickle']
# test_patients = patients[26:29]
test_patients = ['TCGA-08-0509.pickle', 'TCGA-08-0520.pickle', 'TCGA-06-6389.pickle']
image_width = 256
image_height = 256
# batch_size= 16
current_dir = '/home/omar/Desktop/Generalization/Model/'

In [30]:
# Helper Functions
def save_images(path, patient_dataset, mode):
    if mode == 'train':
        os.chdir('/home/omar/Desktop/Generalization/Model/images/train/')
    elif mode == 'test':
        os.chdir('/home/omar/Desktop/Generalization/Model/images/test/')
    for patient in patient_dataset:
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        for scan in range(len(patient_data)):
            if scan != 4:
                for array in range(len(patient_data[scan])):
                    os.chdir('/home/omar/Desktop/Generalization/Model/images/train/scans/')
                    scipy.misc.imsave('%s-%d.png'%(patient,array), patient_data[scan][array])
                    os.chdir('/home/omar/Desktop/Generalization/Model/images/train/labels/')
                    out_image = patient_data[4][scan][array] * 255.0
                    scipy.misc.imsave('%s-%d.png'%(patient,array), out_image)
    if mode=='train':
        print('Training images and labels created.')
    else:
        print('Testing images created.')
    return


def data_gen(path, mode):
    if mode == 'train':
        image_datagen = ImageDataGenerator()
        
        label_datagen = ImageDataGenerator(rotation_range=20,
                                           horizontal_flip=True,
                                           vertical_flip=True,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           shear_range=0.2,
                                           zoom_range=0.1)
        
    elif mode == 'test' or mode == 'valid':
        shuffle=False
        image_datagen = ImageDataGenerator()
        label_datagen = ImageDataGenerator()
    else:
        raise ValueError('dataset mode ERROR!')
    
    image_generator1 = image_datagen.flow_from_directory(
        '/home/omar/Desktop/Generalization/Model/images/train/scans/',
#         classes = [args.image_folder1],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = 16,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        shuffle = shuffle,
        seed = seed)

    label_generator1 = label_datagen.flow_from_directory(
        '/home/omar/Desktop/Generalization/Model/images/train/labels/',
#         classes = [args.label_folder1],
        class_mode = None,
        color_mode = label_color_mode,
        target_size = target_size,
        batch_size = 16,
        save_to_dir = save_to_dir,
        save_prefix  = label_save_prefix,
        shuffle = shuffle,
        seed = seed)
    data_generator1 = zip(image_generator1, label_generator1)
    cnt = 0
    
    if mode == 'test' or mode == 'valid':
#         print('WIP')
        for (img,label) in data_generator1:
            img,label = adjustData(img, label, 'complete', cnt)
            cnt+=img.shape[0]
            yield (img,label)

    else:
        image_generator2 = image_datagen.flow_from_directory(
            '/home/omar/Desktop/Generalization/Model/images/train/scans/',
#             classes = [args.image_folder2],
            class_mode = None,
            color_mode = image_color_mode,
            target_size = target_size,
            batch_size = 16,
            save_to_dir = save_to_dir,
            save_prefix  = image_save_prefix,
            shuffle = shuffle,
            seed = seed+1)

        label_generator2 = label_datagen.flow_from_directory(
            '/home/omar/Desktop/Generalization/Model/images/train/labels/',
#             classes = [args.label_folder2],
            class_mode = None,
            color_mode = label_color_mode,
            target_size = target_size,
            batch_size = 16,
            save_to_dir = save_to_dir,
            save_prefix  = label_save_prefix,
            shuffle = shuffle,
            seed = seed+1)
        data_generator2 = zip(image_generator2, label_generator2)

        while(True):
            if np.random.randint(3) == 2:
                for (img,label) in data_generator1:
                    img,label = adjustData(img, label, 'complete', cnt, 'F')
                    cnt+=img.shape[0]
                    yield (img,label)

            else:
                for (img,label) in data_generator2:
                    img,label = adjustData(img, label, 'complete', cnt, 'S')
                    cnt+=img.shape[0]
                    yield (img,label)

In [31]:
# save_images(sampled_pickles, training_patients, 'train')

In [32]:
# Model
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def UNet():
    inputs = Input((image_width, image_height, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
#     Second Implementation
#     model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

    return model

In [33]:
trainset = data_gen(mode='train')

In [34]:
testset = data_gen(mode='test')

In [35]:
model = UNet()
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
model.fit_generator(trainset, steps_per_epoch=500, shuffle=True, epochs=20,
                        validation_data=testset, validation_steps=2000,
                        callbacks=[model_checkpoint], workers=16)

Epoch 1/20
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ZeroDivisionError: integer division or modulo by zero